In [2]:
# produce graph A

import numpy as np
nodes = []
with open('facebook/0.edges') as f:
    for line in f:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes.append(int(pair[0]))
        nodes.append(int(pair[1]))
    
A_size = max(nodes)+1
A = np.zeros((A_size,A_size))



for i in range(0,len(nodes),2):
    A[nodes[i]][nodes[i+1]]=1
    A[nodes[i+1]][nodes[i]]=1

print(len(A))
    
A[0,:]=1
A[:,0]=1
A[0,0]=0


348


In [3]:
# produce permuted B with noise
def permutation(A):

    perm = np.random.permutation(len(A))

# Compare with zero columns and rows ??

    P = np.identity(len(A))
    P = P[perm]

    B = P.dot(A).dot(P.T)

    noise = np.random.choice([0, 1], size=(len(A),len(A)), p=[99.9/100, 0.1/100])

    B = (B + noise + noise.T)%2

    print("noise edges: " + str(np.sum(np.abs(A-P.T.dot(B).dot(P))[:])))
    
    return B


In [4]:
# Try contructing bigger graph


nodes2 = []



with open('facebook/0.edges') as f1, open('facebook/348.edges') as f2, open('facebook/414.edges') as f3, open('facebook/686.edges') as f4:
    for line in f1:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes2.append(int(pair[0]))
        nodes2.append(int(pair[1]))
    for line in f2:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes2.append(int(pair[0]))
        nodes2.append(int(pair[1]))
    for line in f3:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes2.append(int(pair[0]))
        nodes2.append(int(pair[1]))
    for line in f4:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes2.append(int(pair[0]))
        nodes2.append(int(pair[1]))

A2_size = max(nodes2) + 1
print(A2_size)
A2 = np.zeros((A2_size, A2_size))

for i in range(0, len(nodes2), 2):
    A2[nodes2[i]][nodes2[i+1]]=1
    A2[nodes2[i+1]][nodes2[i]]=1

    
# Add edges to node 0

A2[:348, 0] = 1
A2[0, :348] = 1
A2[0, 0] = 0


print('No connections nodes:')
for i in range(0, A2.shape[0]):
    if sum(A2[i, :]) == 0:
        print(i)




857
No connections nodes:
358
447
550
585
602
607
608
613
624
638
668
674
686
692
801


In [5]:
# Produce different size S1, S2

perm_node = np.random.permutation(A2.shape[0])
#perm_feature = np.random.permutation(A2.shape[1])

node_size = 500
#feature_size = 200

S1 = A2[perm_node[:node_size], :][:, perm_node[:node_size]]



print(S1.shape)




(500, 500)


In [6]:
# Multiple graphs

M1 = A2[:280, :]
M2 = A2[280:560, :]
M3 = A2[560:840, :]

In [7]:
# Node Features
# Nodes Overlapping 
A2_feats = np.zeros((A2_size, 224))

with open('facebook/0.feat') as f1:
    for line in f1:
        feats = line.strip().split()
        A2_feats[int(feats[0])] = feats[1:]

print(A2_feats.shape)

(857, 224)


In [69]:
# Permutation

permutation(A)
permutation(A2)

noise edges: 228.0
noise edges: 1372.0


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [9]:
def minhash(data, hashfuncs):
    '''
        see mining-of-massive-datasets.pdf ch3 minhash for detail
    '''

    #DEBUG = True
    DEBUG = False

    rows, cols, sigrows = len(data), len(data[0]), len(hashfuncs)

    # fucking the shadow copy
    # sigmatrix = [[1000000] * cols] * sigrows
    sigmatrix = []
    for i in range(sigrows):
        sigmatrix.append([1000000] * cols)

    for r in range(rows):
        hashvalue = list(map(lambda x: x(r), hashfuncs))
        if DEBUG: print(hashvalue)
        for c in range(cols):
            if DEBUG: print('-' * 2, r, c)
            if data[r][c] == 0:
                continue
            for i in range(sigrows):
                if DEBUG: print('-' * 4, i, sigmatrix[i][c], hashvalue[i])
                if sigmatrix[i][c] > hashvalue[i]:
                    sigmatrix[i][c] = hashvalue[i]
                if DEBUG: print('-' * 4, sigmatrix)

        if DEBUG:
            for xxxxxxx in sigmatrix:
                print(xxxxxxx)
            print('=' * 30)

    return sigmatrix





if __name__ == '__main__':
    def hash1(x):
        return (x + 1) % 5

    def hash2(x):
        return (3 * x + 1) % 5

    data = [[1, 0, 0, 1],
            [0, 0, 1, 0],
            [0, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 0, 1, 0]]

    print(minhash(data, [hash1, hash2]))

[[1, 3, 0, 1], [0, 2, 0, 0]]


In [37]:
import collections

hashdata = []

print(A_feats.shape)

for i in range(0, A_feats.shape[1]):
    hashdata.append(A_feats[:, i].tolist())

def hash3(x):
    return (500 * x + 300) % 573
def hash4(x):
    return (300* x + 100) % 573
table = minhash(hashdata, [hash3, hash4])

similar =[]
K=[]
for i in range(len(table[0])):
    K.append((table[0][i], table[1][i])) 
        


somelist = K
dups = collections.defaultdict(list)
for index, item in enumerate(somelist):
    dups[item].append(index)



(348, 224)
